In [2]:
from keras.models import Sequential
from keras.layers import Conv2D,MaxPool2D,Activation,Dropout,Flatten,Dense
from keras.preprocessing.image import ImageDataGenerator,img_to_array,load_img
from PIL import Image,ImageFilter
import matplotlib.pyplot as plt
import numpy as np
import keras
import cv2
import os
import tensorflow as tf

Using TensorFlow backend.
/home/prj201904037/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/prj201904037/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/prj201904037/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/prj201904037/miniconda3/lib/p

In [3]:

os.environ["CUDA_VISIBLE_DEVICES"] = "2" #Titan XP
config = tf.ConfigProto()  
config.gpu_options.allow_growth=True   #不全部占满显存, 按需分配
sess = tf.Session(config=config)


In [4]:
#加载检测模型
detect_model = keras.models.load_model("../../model/MandS_Conv3_512_mp2by2_ker4by4_Adam_epoch20.h5")
detect_model.summary()








Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 255, 255, 100)     4900      
_________________________________________________________________
activation_1 (Activation)    (None, 255, 255, 100)     0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 127, 127, 100)     0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 62, 62, 100)       160100    
_________________________________________________________________
activation_2 (Activation)    (None, 62, 62, 100)       0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 31, 31, 100)       0         
_________________________________________

In [6]:
#加载识别模型
recognize_model = keras.models.load_model("../../model/Gray_Conv4_512_mp2by2_ker4by4_Adam_epoch40.h5")
recognize_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_14 (Conv2D)           (None, 509, 509, 130)     2210      
_________________________________________________________________
activation_21 (Activation)   (None, 509, 509, 130)     0         
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 254, 254, 130)     0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 251, 251, 130)     270530    
_________________________________________________________________
activation_22 (Activation)   (None, 251, 251, 130)     0         
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 125, 125, 130)     0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 122, 122, 130)     270530    
__________

In [ ]:
#①分割：用opencv读取进来进行处理，然后进行增强，然后进行分割，分割结果转换为PIL图片
#注意cv2用的是BGR，PIL用的是RGB
def segmentation(imgName):
    origin_img = cv2.imread(imgName)
    

#   复制边缘进行padding
    origin_img = cv2.copyMakeBorder(origin_img,100,100,100,100,cv2.BORDER_REPLICATE)   
    img = origin_img.copy()
    
    
#   增亮度
#     M=np.ones(img.shape,dtype='uint8')*80
#     img=cv2.add(M,img)

#   转灰度
    img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    #gray = cv2.equalizeHist(gray)
    #gray = cv2.normalize(gray,dst=None,alpha=350,beta=10,norm_type=cv2.NORM_MINMAX)

    
    # #先获取白色背景

    # gray_values = []
    # for i in range(gray.shape[0]):
    #     for j in range(gray.shape[1]):
    #         gray_values.append(gray[i][j])
    # gray_values.sort()
    # mid_value = gray_values[int(len(gray_values)/2)]
    # print(mid_value)

    # for i in range(gray.shape[0]):
    #     for j in range(gray.shape[1]):
    #         if gray[i][j]>mid_value:
    #             gray[i][j] = 255
    #         else:
    #             gray[i][j] = 0

    #转灰度 高糊
    img = cv2.GaussianBlur(img, (9, 9),0) 
    ret,blurred = cv2.threshold(img, 0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)

    img = cv2.GaussianBlur(img, (9, 9),0) 
    ret,img = cv2.threshold(img, 0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)

    img = cv2.GaussianBlur(img, (9, 9),0) 
    ret,img = cv2.threshold(img, 0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)

    #plt.imshow(img)

    # #提取梯度
    # gradX = cv2.Sobel(blurred, ddepth=cv2.CV_32F, dx=1, dy=0)
    # gradY = cv2.Sobel(blurred, ddepth=cv2.CV_32F, dx=0, dy=1)

    # gradient = cv2.subtract(gradX, gradY)
    # gradient = cv2.convertScaleAbs(gradient)

    # plt.imshow(gradient)


    #blurred = cv2.GaussianBlur(gradient, (9, 9),0)
    #(_, img) = cv2.threshold(img, 0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    #plt.imshow(thresh)

    #腐蚀膨胀的核
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    img = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)
    #plt.imshow(closed)

    #腐蚀膨胀
    img = cv2.erode(img, None, iterations=2)
    img = cv2.dilate(img, None, iterations=2)
    #plt.imshow(closed)

    #分割
    (_,contours,_) = cv2.findContours(img,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE,)
    #print(len(contours))

    #在原图上裁出分割结果
    segments = []
    for c in contours:
        #todo:阈值选取更加合理一点
        if cv2.arcLength(c, True) >500 and cv2.arcLength(c, True) < 5000 :
            (x, y, w, h) = cv2.boundingRect(c)
            
            #看效果用
            #cv2.rectangle(origin_img, (x, y), (x+w, y+h), (255, 0, 0), 8)
            crop_img= origin_img[y:y+h, x:x+w]
            #cv2.imwrite('../../data/imgNew/imgFromSeg/'+str(count)+".jpg",crop_img) 
            
            #转为PIL图片
            crop_img = Image.fromarray(cv2.cvtColor(crop_img,cv2.COLOR_BGR2RGB))
            segments.append(crop_img)
    return segments

In [ ]:
#②弱分类
#判断为煤返回m，煤矸石返回s，其他返回n
def recognize(origin_img):
    img = origin_img.copy()
    img = img.resize((imgSize,imgSize),Image.ANTIALIAS)
    #img = img.filter(ImageFilter.EDGE_ENHANCE)
    #img = img.filter(ImageFilter.DETAIL)
    img = img.filter(ImageFilter.RankFilter(5, 8))
    img_array_256 = np.array(img).reshape((1,256,256,3))/255
    isObject = detect_model.predict(img_array)[0]
    #是煤或煤矸石
    if isObject>=0.5:
        isisrecognize_model
        pred_value = detect_model.predict(np.array(img).reshape((1,512,512,3))/255)[0]

In [ ]:
#③强分类
from PIL import  ImageFilter
imgSize = 512
#已经拿到了检测-弱分类后的结果，进行强识别，获得煤或者煤矸石的判断
def recognize(origin_img):
    img=img.resize((imgSize,imgSize),Image.ANTIALIAS)
    
    

In [1]:
import os
os.system("ls > names.txt")

0